# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo
from pymongo import MongoClient
client = MongoClient()

Note: you may need to restart the kernel to use updated packages.


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
# Your Code
client = MongoClient('mongodb://localhost:27017')

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [17]:
# Your Code
query={'number_of_employees': {'$gt': 5000}}


select={'name': True,'number_of_employees':True, '_id': False}



list(client.Companies.Pepito.find(query, select).sort('number_of_employees',-1).limit(2))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [23]:
# Your Code
query={'$and':[{'founded_year': {'$gt': 1999}},
               {'founded_year': {'$lt': 2006}}]}


select={'name': True,'founded_year':True, '_id': False}



list(client.Companies.Pepito.find(query, select).limit(5).sort('founded_year',-1))
#Con el sorted podéis mirar que están todas

[{'name': 'MeeVee', 'founded_year': 2000},
 {'name': 'Steorn', 'founded_year': 2000},
 {'name': 'Boonex', 'founded_year': 2000},
 {'name': 'PhotoBox', 'founded_year': 2000},
 {'name': 'AllofMP3', 'founded_year': 2000}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [26]:
# Your Code
query={'$and':[{'ipo.valuation_amount': {'$gt': 1e8}},
               {'founded_year': {'$lt': 2011}}]}


select={'name': True,'ipo.valuation_amount':True, '_id': False} 



list(client.Companies.Pepito.find(query, select).limit(5))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [27]:
# Your Code
query={'$and':[{'number_of_employees': {'$lt': 1000}},
               {'founded_year': {'$lt': 2006}}]}


select={'name': True,'founded_year':True, 'number_of_employees':True, '_id': False}



list(client.Companies.Pepito.find(query, select).limit(10).sort('number_of_employees',-1))

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'Box', 'number_of_employees': 950, 'founded_year': 2005},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Workday', 'number_of_employees': 900, 'founded_year': 2005},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Mozilla', 'number_of_employees': 800, 'founded_year': 1998}]

### 6. All the companies that don't include the `partners` field.

In [32]:
# Your Code
query={ "partners": { '$exists': True } }


select={'name': True, '_id': False}



list(client.Companies.Pepito.find(query, select).limit(5))
#todos tienen partners, lo miré como está, también de otras dos maneras y o lo estoy haciendo fatal o realmente
#sale vacío porque todos tienen la key

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [33]:
# Your Code
query={ "category_code": None }


select={'name': True, '_id': False}



list(client.Companies.Pepito.find(query, select).limit(5))


[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [35]:
# Your Code
query={'$and':[{'number_of_employees': {'$gt': 99}},
               {'number_of_employees': {'$lt': 1000}}]}


select={'name': True,'number_of_employees':True, '_id': False}



list(client.Companies.Pepito.find(query, select).limit(5).sort('number_of_employees',-1))
#Una vez más con el sort podéis mirar que están todos

[{'name': 'Datamonitor', 'number_of_employees': 984},
 {'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931}]

### 9. Order all the companies by their IPO price in a descending order.

In [52]:
# Your Code
select={'name': True,'ipo.valuation_amount':True, '_id': False} 



list(client.Companies.Pepito.find({},select).limit(5).sort('ipo.valuation_amount',-1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [55]:
# Your Code
select={'name': True,'number_of_employees':True, '_id': False} 



list(client.Companies.Pepito.find(query, select).sort('number_of_employees',-1).limit(10))

[{'name': 'Datamonitor', 'number_of_employees': 984},
 {'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Relax Solutions Pvt Ltd', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'InMobi', 'number_of_employees': 900}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [56]:
# Your Code
query={'founded_month': {'$lt': 7}}
            

select={'name': True,'founded_month':True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(1000))

[{'name': 'Facebook', 'founded_month': 2},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'Twitter', 'founded_month': 3},
 {'name': 'StumbleUpon', 'founded_month': 2},
 {'name': 'Gizmoz', 'founded_month': 1},
 {'name': 'Helio', 'founded_month': 2},
 {'name': 'Babelgum', 'founded_month': 3},
 {'name': 'Yahoo!', 'founded_month': 1},
 {'name': 'Technorati', 'founded_month': 6},
 {'name': 'OpenX', 'founded_month': 5},
 {'name': 'Mahalo', 'founded_month': 3},
 {'name': 'Sparter', 'founded_month': 6},
 {'name': 'Meetup', 'founded_month': 1},
 {'name': 'Jangl SMS', 'founded_month': 3},
 {'name': 'Prosper', 'founded_month': 2},
 {'name': 'Jajah', 'founded_month': 1},
 {'name': 'YouTube', 'founded_month': 2},
 {'name': 'Stickam', 'founded_month': 2},
 {'name': 'blogTV', 'founded_month': 5},
 {'name': 'Livestream', 'founded_month': 5},
 {'name': 'GrandCentral', 'founded_month': 4},
 {'name': 'Topi

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [60]:
# Your Code
query={'$and':[{'founded_year': {'$lt': 2000}},{'acquisition.price_amount': {'$gt': 10000000}}]}
            

select={'name': True,'founded_year':True, 'acquisition.price_amount': True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(5))

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [61]:
# Your Code
query={'acquisition.acquired_year': {'$gt': 2010}}
            

select={'name': True,'acquisition': True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(5).sort('acquisition.price_amount',-1))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [62]:
# Your Code         

select={'name': True,'founded_year': True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(5).sort('founded_year',-1))


[{'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Ptch', 'founded_year': 2012},
 {'name': 'travelmob', 'founded_year': 2011},
 {'name': 'Social Gaming Network', 'founded_year': 2011},
 {'name': 'travelmob', 'founded_year': 2011}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [63]:
# Your Code
query={'founded_day': {'$lt': 8}}
            

select={'name': True,'founded_day': True,'acquisition.price_amount': True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(10).sort('acquisition.price_amount',-1))

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [67]:
# Your Code
query={'$and':[{'category_code': 'web'},
               {'number_of_employees': {'$gt': 4000}}]}
            

select={'name': True,'category_code': True,'number_of_employees':True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(5).sort('number_of_employees',1))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [68]:
# Your Code
query={'$and':[{'acquisition.price_currency_code':'EUR'},{'acquisition.price_amount': {'$gt': 10000000}}]}
            

select={'name': True, 'acquisition.price_amount': True,'acquisition.price_currency_code':True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(5))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [69]:
# Your Code
query={'acquisition.acquired_month': {'$lt': 4}}
            

select={'name': True,'acquisition':True, '_id': False} 


list(client.Companies.Pepito.find(query, select).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code